# Building a ReAct Agent from Scratch

This notebook implements a ReAct (Reasoning + Acting) agent using AWS Bedrock.

The agent follows this pattern:
1. **Thought**: Reasons about what to do
2. **Action**: Decides an action to take
3. **Observation**: Receives result from action
4. **Repeat**: Continues until task is complete

In [ ]:
# Install required packages
!pip install boto3 -q

In [ ]:
# Import Google Colab userdata for secure credential access
from google.colab import userdata
import boto3
import json

# Configure your AWS credentials using Colab secrets
AWS_ACCESS_KEY_ID = userdata.get('awsid')  # Set this in Colab secrets
AWS_SECRET_ACCESS_KEY = userdata.get('awssecret')  # Set this in Colab secrets
AWS_REGION = "us-east-1"  # Change if needed

# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

print("✓ AWS Bedrock client initialized")

In [ ]:
# Import additional libraries
import re
from typing import List, Dict, Any

In [ ]:
# Test the Bedrock client with a simple message
def call_bedrock(messages: List[Dict[str, str]], system: str = "") -> str:
    """
    Call AWS Bedrock Claude model with messages.
    """
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature": 0,
        "messages": messages
    }
    
    if system:
        body["system"] = system
    
    response = bedrock_runtime.invoke_model(
        modelId="anthropic.claude-3-5-sonnet-20241022-v2:0",
        body=json.dumps(body)
    )
    
    response_body = json.loads(response['body'].read())
    return response_body['content'][0]['text']

# Test it
test_response = call_bedrock([{"role": "user", "content": "Hello, world!"}])
print(test_response)

## Define the Agent Class

The agent maintains a conversation history and executes the ReAct loop.

In [ ]:
class Agent:
    def __init__(self, system: str = ""):
        """
        Initialize the agent with an optional system message.
        """
        self.system = system
        self.messages: List[Dict[str, str]] = []
    
    def __call__(self, message: str) -> str:
        """
        Process a message and return the agent's response.
        """
        # Add user message to conversation history
        self.messages.append({"role": "user", "content": message})
        
        # Get response from the model
        result = self.execute()
        
        # Add assistant message to conversation history
        self.messages.append({"role": "assistant", "content": result})
        
        return result
    
    def execute(self) -> str:
        """
        Execute the model call with current messages.
        """
        return call_bedrock(self.messages, self.system)

print("✓ Agent class defined")

## Create the ReAct System Prompt

This prompt instructs the model how to use the Thought-Action-Observation loop.

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.

Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dog's weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weighs 51 lbs

You then output:

Answer: A Bulldog weighs 51 lbs
""".strip()

print("✓ System prompt defined")
print("\nPrompt preview:")
print(prompt[:200] + "...")

## Define Available Tools/Actions

These are the functions the agent can call during execution.

In [ ]:
def calculate(operation: str) -> float:
    """
    Perform a mathematical calculation.
    """
    return eval(operation)

def average_dog_weight(breed: str) -> str:
    """
    Return the average weight of a dog breed.
    This is a mock implementation with hardcoded values.
    """
    breed = breed.strip().lower()
    
    weights = {
        'scottish terrier': '20 lbs',
        'border collie': '37 lbs',
        'toy poodle': '7 lbs'
    }
    
    return weights.get(breed, f"Unknown breed: {breed}")

# Dictionary mapping action names to functions
known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

print("✓ Tools defined:")
for action in known_actions.keys():
    print(f"  - {action}")

## Manual Agent Execution (Step-by-step)

Let's first run the agent manually to understand each step.

In [ ]:
# Initialize agent with the ReAct prompt
agent = Agent(system=prompt)

# Step 1: Ask the question
result = agent("How much does a toy poodle weigh?")
print("Step 1 - Agent's first response:")
print(result)
print("\n" + "="*50 + "\n")

In [ ]:
# Step 2: Execute the action manually
# Parse the action from the response
action_match = re.search(r'Action: ([a-z_]+): (.+)', result, re.IGNORECASE)
if action_match:
    action_name = action_match.group(1).strip()
    action_input = action_match.group(2).strip()
    
    print(f"Executing: {action_name}({action_input})")
    
    # Call the function
    observation = known_actions[action_name](action_input)
    print(f"Observation: {observation}")
    
    # Format the observation for the agent
    next_prompt = f"Observation: {observation}"
    print("\n" + "="*50 + "\n")

In [ ]:
# Step 3: Pass observation back to agent
result = agent(next_prompt)
print("Step 3 - Agent's final response:")
print(result)
print("\n" + "="*50 + "\n")

In [ ]:
# View the complete message history
print("Complete conversation history:")
for i, msg in enumerate(agent.messages):
    print(f"\n{i+1}. {msg['role'].upper()}:")
    print(msg['content'][:200] + ("..." if len(msg['content']) > 200 else ""))

## Complex Example (Manual)

Let's try a more complex question that requires multiple steps.

In [ ]:
# Reinitialize the agent
agent = Agent(system=prompt)

# Ask a complex question
question = "I have two dogs, a border collie and a Scottish terrier. What is their combined weight?"
result = agent(question)
print("Question:", question)
print("\nAgent response:")
print(result)
print("\n" + "="*50)

## Automated Agent Loop

Now let's automate the entire ReAct loop.

In [ ]:
# Regex pattern to find actions in the response
action_re = re.compile(r'^Action: (\w+): (.*)$', re.MULTILINE)

def query(question: str, max_turns: int = 5) -> str:
    """
    Run the agent in an automated loop until it provides an answer.
    """
    i = 0
    agent = Agent(system=prompt)
    next_prompt = question
    
    while i < max_turns:
        i += 1
        print(f"\n{'='*60}")
        print(f"Turn {i}")
        print(f"{'='*60}")
        
        # Get agent's response
        result = agent(next_prompt)
        print(result)
        
        # Parse for actions
        actions = [
            (action_match.group(1), action_match.group(2))
            for action_match in action_re.finditer(result)
        ]
        
        # If there are actions to take
        if actions:
            action, action_input = actions[0]
            
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            
            print(f"\n→ Executing: {action}({action_input})")
            observation = known_actions[action](action_input)
            print(f"→ Observation: {observation}")
            
            next_prompt = f"Observation: {observation}"
        else:
            # No actions found, agent is done
            print("\n✓ Agent finished!")
            return result
    
    print("\n⚠ Max turns reached")
    return result

print("✓ Query function defined")

## Run the Automated Agent

In [ ]:
# Simple question
result = query("How much does a toy poodle weigh?")

In [ ]:
# Complex question requiring multiple steps
result = query("I have two dogs, a border collie and a Scottish terrier. What is their combined weight?")

In [ ]:
# Question requiring calculation
result = query("What is 37 multiplied by 42, divided by 3?")

## Summary

In this notebook, we built a ReAct agent from scratch that:

1. **Thinks** about problems using natural language reasoning
2. **Acts** by calling available tools/functions
3. **Observes** the results of those actions
4. **Repeats** until it can provide a final answer

Key components:
- **Agent Class**: Manages conversation history and model calls
- **System Prompt**: Instructs the model on the ReAct pattern
- **Tools**: Functions the agent can call (calculate, average_dog_weight)
- **Query Loop**: Automates the Thought-Action-Observation cycle

This is a foundation for building more sophisticated agents with additional tools and capabilities!